In [1]:
import sqlite3

In [2]:
type(sqlite3)

module

In [3]:
conn = sqlite3.connect("first.db")

In [4]:
# those kind of databases live in memory and vanish while restarting a computer or somethingb
in_mem = sqlite3.connect(":memory:")

In [4]:
cur = conn.cursor()

In [6]:
cur.execute("""
            CREATE TABLE ice_cream_flavors (
            id INTEGER PRIMARY KEY,
            flavor TEXT,
            rating INTEGER
            );
            """)

OperationalError: table ice_cream_flavors already exists

In [7]:
# It only stages data
# to insert it you have to commit
cur.execute("""
            INSERT INTO ice_cream_flavors (flavor, rating)
            VALUES ('choco', 10), ('straw', 9), ('pistachio', 7.2);
            """)

In [9]:
conn.commit()

In [10]:
cur.execute("SELECT * FROM ice_cream_flavors;")

In [11]:
cur.fetchone()

(1, 'choco', 10)

In [12]:
cur.fetchone()

(2, 'straw', 9)

In [13]:
next(cur)

(3, 'pistachio', 7.2)

In [14]:
# it is equwalent to fetchone
# lazy evaluation
cur.execute("SELECT * FROM ice_cream_flavors;")

for row in cur:
  print(row)

(1, 'choco', 10)
(2, 'straw', 9)
(3, 'pistachio', 7.2)
(4, 'choco', 10)
(5, 'straw', 9)
(6, 'pistachio', 7.2)


In [15]:
# it gives a list of tuples
cur.execute("SELECT * FROM ice_cream_flavors;")

cur.fetchall()

[(1, 'choco', 10),
 (2, 'straw', 9),
 (3, 'pistachio', 7.2),
 (4, 'choco', 10),
 (5, 'straw', 9),
 (6, 'pistachio', 7.2)]

In [16]:
cur.execute("SELECT * FROM ice_cream_flavors;")
cur.fetchmany(2)

[(1, 'choco', 10), (2, 'straw', 9)]

In [17]:
cur.row_factory = None
# - list of column names
# - seq of records

In [18]:
cur.description

(('id', None, None, None, None, None, None),
 ('flavor', None, None, None, None, None, None),
 ('rating', None, None, None, None, None, None))

In [19]:
cur.row_factory = lambda cursor, row: dict(zip([col[0] for col in cursor.description], row))

In [20]:
cur.execute("SELECT * FROM ice_cream_flavors;")

result = cur.fetchone()

result['flavor']

'choco'

In [21]:
result['rating']

10

In [22]:
cur.execute("SELECT * FROM ice_cream_flavors;")

result = cur.fetchall()

result

[{'id': 1, 'flavor': 'choco', 'rating': 10},
 {'id': 2, 'flavor': 'straw', 'rating': 9},
 {'id': 3, 'flavor': 'pistachio', 'rating': 7.2},
 {'id': 4, 'flavor': 'choco', 'rating': 10},
 {'id': 5, 'flavor': 'straw', 'rating': 9},
 {'id': 6, 'flavor': 'pistachio', 'rating': 7.2}]

In [23]:
sqlite3.Row

sqlite3.Row

In [24]:
cur.row_factory = None

In [25]:
cur.execute("SELECT * FROM ice_cream_flavors;")

result = cur.fetchall()

result

[(1, 'choco', 10),
 (2, 'straw', 9),
 (3, 'pistachio', 7.2),
 (4, 'choco', 10),
 (5, 'straw', 9),
 (6, 'pistachio', 7.2)]

In [26]:
cur.row_factory = sqlite3.Row

In [27]:
cur.execute("SELECT * FROM ice_cream_flavors;")

result = cur.fetchall()

result

In [28]:
first_row = result[0]

In [29]:
first_row["flavor"]

'choco'

In [30]:
first_row["rating"]

10

In [31]:
first_row["id"]

1

In [32]:
first_row[0]

1

In [33]:
# to unstage insert use rollback() 
cur.execute("""
            INSERT INTO ice_cream_flavors (flavor, rating)
            VALUES ('hazelnut', 6);
            """)

In [34]:
conn.rollback()

In [40]:
# to insert many rows use dict
flavor = input("Enter a flavor: ")
rating = input("Enter a rating: ")

cur.execute(f"INSERT INTO ice_cream_flavors (flavor, rating) VALUES (:flavor, :rating);",
            {
              "flavor": flavor,
              "rating": rating
            })
conn.commit()

In [42]:
# to insert one row
cur.execute(f"INSERT INTO ice_cream_flavors (flavor, rating) VALUES (?, ?);", 
            ('fudge', 6.7))
conn.commit()

In [44]:
ice_cream_flavors = [ 
  ('vanilla', 4.5), 
  ('strawberry', 9), 
  ('mint', 3.6), 
  ('coffee', 6.0),
  ('caramel', 8), 
  ('saltedcaramel', 9.2),
]

In [46]:
# to insert a few rows
cur.executemany(f"INSERT INTO ice_cream_flavors (flavor, rating) VALUES (?, ?);",
            ice_cream_flavors)
conn.commit()

In [48]:
# script to create table and insert values
cur.executescript("""
CREATE TABLE IF NOT EXISTS toppings (
  id INTEGER PRIMARY KEY,
  name TEXT NOT NULL
);

INSERT INTO toppings (name) VALUES ('sprinkles'), ('choco sauce'), ('carmel sauce'), ('nuts');
""")

In [49]:
conn.commit()

In [5]:
obj = conn.iterdump()

In [6]:
next(obj)

'BEGIN TRANSACTION;'

In [7]:
for line in conn.iterdump():
  print(line)

BEGIN TRANSACTION;
CREATE TABLE ice_cream_flavors (
            id INTEGER PRIMARY KEY,
            flavor TEXT,
            rating INTEGER
            );
INSERT INTO "ice_cream_flavors" VALUES(1,'choco',10);
INSERT INTO "ice_cream_flavors" VALUES(2,'straw',9);
INSERT INTO "ice_cream_flavors" VALUES(3,'pistachio',7.2);
INSERT INTO "ice_cream_flavors" VALUES(4,'hazelnut',6.3);
INSERT INTO "ice_cream_flavors" VALUES(5,'cherries',4.3);
INSERT INTO "ice_cream_flavors" VALUES(6,'bannana',2.1);
INSERT INTO "ice_cream_flavors" VALUES(7,'fudge',6.7);
INSERT INTO "ice_cream_flavors" VALUES(8,'vanilla',4.5);
INSERT INTO "ice_cream_flavors" VALUES(9,'strawberry',9);
INSERT INTO "ice_cream_flavors" VALUES(10,'mint',3.6);
INSERT INTO "ice_cream_flavors" VALUES(11,'coffee',6);
INSERT INTO "ice_cream_flavors" VALUES(12,'caramel',8);
INSERT INTO "ice_cream_flavors" VALUES(13,'saltedcaramel',9.2);
CREATE TABLE toppings (
  id INTEGER PRIMARY KEY,
  name TEXT NOT NULL
);
INSERT INTO "toppings" VALUES(1,

In [6]:
# how to save iterdump
with open("ice_cream_db_dump.sel", "w") as _file:
  for line in conn.iterdump():
    _file.write(f"{line}\n")

In [5]:
with conn:
  cur.execute(f"INSERT INTO ice_cream_flavors (flavor, rating) VALUES (?, ?);", 
            ('super_salted_carmel', 2.7))

In [6]:
conn.close()